In [ ]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
import os
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

Current working directory: /content


# Read in dataset

In [ ]:
df = pd.read_csv("Real estate list.csv")
df2 = pd.read_csv("median incomes.csv")


FileNotFoundError: ignored

In [ ]:
df.head()

,year,month,yyyymm,postal_code,zip_name,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,median_listing_price_per_square_foot,median_square_feet,average_listing_price,total_listing_count,pending_ratio
0,2021,2,202102,10001,"new york, ny",3472500.0,172,147.5,16,0,16,34,1749.437133,1775.0,4836730.989,206,0.197674
1,2021,1,202101,10001,"new york, ny",3285000.0,187,128.0,24,0,16,35,1677.258284,1717.0,4730189.603,222,0.187166
2,2020,12,202012,10001,"new york, ny",2650000.0,206,112.5,12,0,20,25,1566.469094,1682.0,4571220.940,231,0.121359
3,2020,11,202011,10001,"new york, ny",2595000.0,216,97.0,20,0,20,21,1506.233766,1578.0,4444411.123,237,0.097222
4,2020,10,202010,10001,"new york, ny",2750000.0,214,68.0,72,0,40,19,1560.846561,1578.0,4761281.106,233,0.088785


In [ ]:
df2

,Location,Household Type,TimeFrame,DataFormat,Data,Fips
0,10001,Families without Children,2019,Dollars,132545.00000,10001
1,10001,Families without Children,2018,Dollars,122776.87400,10001
2,10001,Families,2019,Dollars,122776.00000,10001
3,10001,Families with Children,2017,Dollars,119270.19390,10001
4,10001,Families,2018,Dollars,115436.24390,10001
...,...,...,...,...,...,...
2163,11697,Families without Children,2018,Dollars,107920.49760,11697
2164,11697,All Households,2019,Dollars,105750.00000,11697
2165,11697,All Households,2018,Dollars,105456.65220,11697
2166,11697,All Households,2017,Dollars,104076.69810,11697


# Data Profiling

In [ ]:
df.shape

(55553, 17)

In [ ]:
df2.shape

(2168, 6)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55553 entries, 0 to 55552
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   year                                  55553 non-null  int64  
 1   month                                 55553 non-null  int64  
 2   yyyymm                                55553 non-null  int64  
 3   postal_code                           55553 non-null  int64  
 4   zip_name                              55553 non-null  object 
 5   median_listing_price                  55553 non-null  float64
 6   active_listing_count                  55553 non-null  int64  
 7   median_days_on_market                 55553 non-null  float64
 8   new_listing_count                     55553 non-null  int64  
 9   price_increased_count                 55553 non-null  int64  
 10  price_reduced_count                   55553 non-null  int64  
 11  pending_listing

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2168 entries, 0 to 2167
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Location        2168 non-null   int64  
 1   Household Type  2168 non-null   object 
 2   TimeFrame       2168 non-null   int64  
 3   DataFormat      2168 non-null   object 
 4   Data            2168 non-null   float64
 5   Fips            2168 non-null   int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 101.8+ KB


In [ ]:
df.describe()

,year,month,yyyymm,postal_code,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,median_listing_price_per_square_foot,median_square_feet,average_listing_price,total_listing_count,pending_ratio
count,55553.000000,55553.000000,55553.000000,55553.000000,5.555300e+04,55553.000000,55553.000000,55553.000000,55553.000000,55553.000000,55553.000000,55553.000000,55553.000000,5.555300e+04,55553.000000,55553.000000
mean,2018.198927,6.772002,201826.664717,12368.417331,4.956892e+05,64.732634,97.637688,14.987489,0.195273,9.577593,19.907206,199.934772,1503.713850,6.669916e+05,84.639839,0.318648
std,1.372738,3.455950,136.438150,1316.254324,5.828141e+05,68.560112,62.776518,18.171787,1.007976,13.363760,38.181344,276.359641,877.307957,9.193808e+05,93.720548,0.490632
min,2016.000000,1.000000,201607.000000,10001.000000,1.500000e+02,13.000000,4.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.102373e+04,13.000000,0.000000
25%,2017.000000,4.000000,201707.000000,11374.000000,1.799000e+05,23.000000,60.500000,4.000000,0.000000,0.000000,1.000000,73.639456,1100.000000,2.196529e+05,29.000000,0.036765
50%,2018.000000,7.000000,201809.000000,12208.000000,3.250000e+05,43.000000,84.000000,8.000000,0.000000,4.000000,6.000000,127.104394,1625.000000,3.905756e+05,55.000000,0.157895
75%,2019.000000,10.000000,201911.000000,13339.000000,5.700000e+05,79.000000,116.500000,20.000000,0.000000,12.000000,21.000000,210.247490,1981.500000,6.875146e+05,102.000000,0.400000
max,2021.000000,12.000000,202102.000000,14905.000000,8.400000e+06,923.000000,968.500000,308.000000,28.000000,236.000000,1140.000000,2458.150576,7506.500000,2.140000e+07,1267.000000,10.458716


In [ ]:
df2.describe()

,Location,TimeFrame,Data,Fips
count,2168.000000,2168.000000,2168.000000,2168.000000
mean,10815.458948,2017.999539,90139.091896,10815.458948
std,571.477656,0.816591,54004.737638,571.477656
min,10001.000000,2017.000000,20114.000000,10001.000000
25%,10302.000000,2017.000000,54481.973150,10302.000000
50%,11105.000000,2018.000000,74618.068145,11105.000000
75%,11360.000000,2019.000000,105396.000000,11360.000000
max,11697.000000,2019.000000,260735.115000,11697.000000


# Data cleansing

In [ ]:
# is there any null value in the dataset
df.isna().any()

year                                    False
month                                   False
yyyymm                                  False
postal_code                             False
zip_name                                False
median_listing_price                    False
active_listing_count                    False
median_days_on_market                   False
new_listing_count                       False
price_increased_count                   False
price_reduced_count                     False
pending_listing_count                   False
median_listing_price_per_square_foot    False
median_square_feet                      False
average_listing_price                   False
total_listing_count                     False
pending_ratio                           False
dtype: bool

In [ ]:
df2.isna().any()

Location          False
Household Type    False
TimeFrame         False
DataFormat        False
Data              False
Fips              False
dtype: bool

In [ ]:
df[df.duplicated()]

,year,month,yyyymm,postal_code,zip_name,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,median_listing_price_per_square_foot,median_square_feet,average_listing_price,total_listing_count,pending_ratio


In [ ]:
df2[df2.duplicated()]

,Location,Household Type,TimeFrame,DataFormat,Data,Fips


# Transformation

## Creating date dimension

In [ ]:
date_dim = df[['year', 'month', 'yyyymm']]
date_dim = date_dim.drop_duplicates()

In [ ]:
#creating date_id values starting from 50000
import random
dateID = []
num = 50000
for i in range(0, len(date_dim)):
    num +=1
    dateID.append(num)

# adding the date_id column to the date_dim
date_dim["date_id"] = dateID

In [ ]:
#reordering to put it in a hierarchical order
date_dim = date_dim[["date_id", "month", "year", "yyyymm"]]
date_dim

,date_id,month,year,yyyymm
0,50001,2,2021,202102
1,50002,1,2021,202101
2,50003,12,2020,202012
3,50004,11,2020,202011
4,50005,10,2020,202010
5,50006,9,2020,202009
6,50007,8,2020,202008
7,50008,7,2020,202007
8,50009,6,2020,202006
9,50010,5,2020,202005


In [ ]:
#adding a month column to the "median income" dataset so that both "real estate" and "median income" datasets'
#date format are the same

month = []
for i in range (0, len(df2)):
    num = 12
    month.append(num)

df2["Month"] = month
df2.rename(columns = {"TimeFrame" : "Year"}, inplace = True)
df2["yyyymm"] = df2["Year"].astype(str) + df2["Month"].astype(str)
df2

,Location,Household Type,Year,DataFormat,Data,Fips,Month,yyyymm
0,10001,Families without Children,2019,Dollars,132545.00000,10001,12,201912
1,10001,Families without Children,2018,Dollars,122776.87400,10001,12,201812
2,10001,Families,2019,Dollars,122776.00000,10001,12,201912
3,10001,Families with Children,2017,Dollars,119270.19390,10001,12,201712
4,10001,Families,2018,Dollars,115436.24390,10001,12,201812
...,...,...,...,...,...,...,...,...
2163,11697,Families without Children,2018,Dollars,107920.49760,11697,12,201812
2164,11697,All Households,2019,Dollars,105750.00000,11697,12,201912
2165,11697,All Households,2018,Dollars,105456.65220,11697,12,201812
2166,11697,All Households,2017,Dollars,104076.69810,11697,12,201712


## Creating location dimension

In [ ]:
#Change column name from RegionName to ZipCode to make it clear
df =df.rename(columns={"zip_name": "citystate"})

#separating the location dimension columns from the main dataset
location_dim = df[['postal_code', "citystate"]]
location_dim = location_dim.drop_duplicates()
location_dim

,postal_code,citystate
0,10001,"new york, ny"
56,10002,"new york, ny"
112,10003,"new york, ny"
168,10004,"new york, ny"
224,10005,"new york, ny"
...,...,...
55274,14895,"wellsville, ny"
55330,14901,"elmira, ny"
55386,14903,"elmira, ny"
55441,14904,"elmira, ny"


In [ ]:
# Creating a hierarchy for the location dimension. Therefore, we separated the citystate column to a city column
# and a state column

location_dim["city"] = location_dim['citystate'].str.rstrip(", ny")

state = []
for i in range (0, len(location_dim)):
    st = "NY"
    state.append(st)
    
location_dim["state"] = state

location_dim = location_dim.drop(columns=["citystate"], axis=1)

location_dim

,postal_code,city,state
0,10001,new york,NY
56,10002,new york,NY
112,10003,new york,NY
168,10004,new york,NY
224,10005,new york,NY
...,...,...,...
55274,14895,wellsville,NY
55330,14901,elmira,NY
55386,14903,elmira,NY
55441,14904,elmira,NY


In [ ]:
#creating location_id for the location dimension

import random
indexes = []
num = 1999
for i in range(0, len(location_dim)):
    num +=1
    indexes.append(num)

location_dim["location_id"] = indexes
location_dim = location_dim[["location_id", "postal_code", "city", "state"]]
location_dim

,location_id,postal_code,city,state
0,2000,10001,new york,NY
56,2001,10002,new york,NY
112,2002,10003,new york,NY
168,2003,10004,new york,NY
224,2004,10005,new york,NY
...,...,...,...,...
55274,3341,14895,wellsville,NY
55330,3342,14901,elmira,NY
55386,3343,14903,elmira,NY
55441,3344,14904,elmira,NY


In [ ]:
location_dim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1346 entries, 0 to 55497
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   location_id  1346 non-null   int64 
 1   postal_code  1346 non-null   int64 
 2   city         1346 non-null   object
 3   state        1346 non-null   object
dtypes: int64(2), object(2)
memory usage: 52.6+ KB


## Creating Property Fact Table

In [ ]:
# creating dataframe with the numerical values that we need
prop_value = df[["median_listing_price", "yyyymm", "average_listing_price", "median_days_on_market", "postal_code"]]
prop_value

,median_listing_price,yyyymm,average_listing_price,median_days_on_market,postal_code
0,3472500.0,202102,4.836731e+06,147.5,10001
1,3285000.0,202101,4.730190e+06,128.0,10001
2,2650000.0,202012,4.571221e+06,112.5,10001
3,2595000.0,202011,4.444411e+06,97.0,10001
4,2750000.0,202010,4.761281e+06,68.0,10001
...,...,...,...,...,...
55548,124900.0,201611,1.648717e+05,132.0,14905
55549,129900.0,201610,1.646102e+05,108.0,14905
55550,133900.0,201609,1.570856e+05,98.5,14905
55551,129900.0,201608,1.588308e+05,81.5,14905


In [ ]:
# adding the date_id to the fact table by merging it with date dimension 
prop_fact = pd.merge(prop_value, date_dim, on="yyyymm")

#add location_id to the fact_table by merging fact_table with location_dim
prop_fact = pd.merge(location_dim, prop_fact, on='postal_code')

#removing columns that shouldn't be in the fact table
prop_fact = prop_fact.drop(columns=["yyyymm", "year", "month", "city","state","postal_code"], axis=1)
prop_fact

,location_id,median_listing_price,average_listing_price,median_days_on_market,date_id
0,2000,3472500.0,4.836731e+06,147.5,50001
1,2000,3285000.0,4.730190e+06,128.0,50002
2,2000,2650000.0,4.571221e+06,112.5,50003
3,2000,2595000.0,4.444411e+06,97.0,50004
4,2000,2750000.0,4.761281e+06,68.0,50005
...,...,...,...,...,...
55548,3345,124900.0,1.648717e+05,132.0,50052
55549,3345,129900.0,1.646102e+05,108.0,50053
55550,3345,133900.0,1.570856e+05,98.5,50054
55551,3345,129900.0,1.588308e+05,81.5,50055


## Creating Household Dimension

In [ ]:
#create dimension for location
dimension_rows = []

#all unique locations
all_household = df2['Household Type'].unique().tolist()

for household_id, household in enumerate(all_household, start = 40000):
    temp_list = [household_id,household]
    dimension_rows.append(temp_list)
    
household_dim = pd.DataFrame(data=dimension_rows,
                         columns = ['household_id','Household Type'])

household_dim.set_index("household_id")

,Household Type
household_id,
40000,Families without Children
40001,Families
40002,Families with Children
40003,All Households


In [ ]:
household_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   household_id    4 non-null      int64 
 1   Household Type  4 non-null      object
dtypes: int64(1), object(1)
memory usage: 192.0+ bytes


## Creating Fact Table Income

In [ ]:
# creating dataframe with the numerical values that we need
income_fact = df2[["Location", "Data", "Household Type","yyyymm"]]
income_fact

,Location,Data,Household Type,yyyymm
0,10001,132545.00000,Families without Children,201912
1,10001,122776.87400,Families without Children,201812
2,10001,122776.00000,Families,201912
3,10001,119270.19390,Families with Children,201712
4,10001,115436.24390,Families,201812
...,...,...,...,...
2163,11697,107920.49760,Families without Children,201812
2164,11697,105750.00000,All Households,201912
2165,11697,105456.65220,All Households,201812
2166,11697,104076.69810,All Households,201712


In [ ]:
#changing the data type for "yyyymm" column to match it with date_dim, which will then allow us to merge the
#dataframe using yyyymm column
income_fact["yyyymm"] = income_fact.yyyymm.astype(int)

<ipython-input-41-c3a6e8f93dd7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_fact["yyyymm"] = income_fact.yyyymm.astype(int)


In [ ]:
#Adding Location ID to the fact table
income_fact = income_fact.merge(location_dim, left_on='Location', right_on='postal_code')

#Adding Household ID to income fact table
income_fact = pd.merge(income_fact, household_dim, on="Household Type")

#Adding Household ID to income fact table
income_fact = pd.merge(income_fact, date_dim, on="yyyymm")

#drop unneeded columns
income_fact = income_fact.drop(columns=["Location", "Household Type", 
                                        "city","postal_code", "state", "yyyymm", "month", "year"], axis=1)

income_fact

,Data,location_id,household_id,date_id
0,132545.00000,2000,40000,50015
1,38639.00000,2001,40000,50015
2,153480.00000,2002,40000,50015
3,204130.00000,2003,40000,50015
4,202042.00000,2004,40000,50015
...,...,...,...,...
2091,72232.72438,2293,40003,50039
2092,44255.95845,2331,40003,50039
2093,47783.16898,2332,40003,50039
2094,59909.38920,2333,40003,50039


In [ ]:
#reordering the columns so that the foreign key columns come before "data" column
income_fact = income_fact[['date_id', 'location_id','household_id','Data']]
income_fact

,date_id,location_id,household_id,Data
0,50015,2000,40000,132545.00000
1,50015,2001,40000,38639.00000
2,50015,2002,40000,153480.00000
3,50015,2003,40000,204130.00000
4,50015,2004,40000,202042.00000
...,...,...,...,...
2091,50039,2293,40003,72232.72438
2092,50039,2331,40003,44255.95845
2093,50039,2332,40003,47783.16898
2094,50039,2333,40003,59909.38920


In [ ]:
income_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2096 entries, 0 to 2095
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date_id       2096 non-null   int64  
 1   location_id   2096 non-null   int64  
 2   household_id  2096 non-null   int64  
 3   Data          2096 non-null   float64
dtypes: float64(1), int64(3)
memory usage: 81.9 KB


# Load

In [ ]:
key_path = '/Users/ericseda/Downloads/nba-project-304419-11fc0536b2d6.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],)

client = bigquery.Client(credentials=credentials, project =credentials.project_id,)

dataset_id = 'nba-project-304419.nba_data'

dataset_ref = client.dataset(dataset_id)
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True
job_config.write_disposition = "WRITE_TRUNCATE"

load_job = client.load_table_from_dataframe(date_dim, "nba_data.date_dim",
                                            job_config=job_config)

print("Starting job {}".format(load_job))
print("Done!", 200)


In [ ]:
load_job = client.load_table_from_dataframe(location_dim, "nba_data.location_dim",
                                            job_config=job_config)

print("Starting job {}".format(load_job))
print("Done!", 300)

In [ ]:
load_job = client.load_table_from_dataframe(prop_fact, "nba_data.prop_fact",
                                            job_config=job_config)

print("Starting job {}".format(load_job))
print("Done!", 400)

In [ ]:
household_dim.rename(columns = {"Household Type" : "Household_Type"}, inplace = True)

load_job = client.load_table_from_dataframe(household_dim, "nba_data.household_dim",
                                            job_config=job_config)

print("Starting job {}".format(load_job))
print("Done!", 500)

In [ ]:
load_job = client.load_table_from_dataframe(income_fact, "nba_data.income_fact",
                                            job_config=job_config)

print("Starting job {}".format(load_job))
print("Done!", 600)